# Demo für Python sqlite3 API

1. Importiere die SQLite 3 API 
2. Importiere die Pathlib (zum Erstellen des Pfad-Objektes)
3. Definiere den Dateipfad

In [1]:
import sqlite3
from pathlib import Path

full_database_path = 'data/presentation.db'

\
Frage die Version der API ab

In [2]:
print(sqlite3.version)

2.6.0


\
Frage die Version der verwendeten SQLite Database Engine ab

In [3]:
print(sqlite3.sqlite_version)

3.32.3


---

## Datenbankverbindung erstellen
1. Initialisiere eine Variable zum einspeichern des Verbindungs-Zustandes
2. Erstelle das Connection-Object und erstelle die Datenbank am definierten Zielort, falls sie nocht nicht existiert

In [34]:
connectionEstablished = False
con = sqlite3.connect(full_database_path)

1. Versuche ein Cursor-Object zu erstellen
 1. Konnte ein Cursor erstellt werden, so wird die Verbindung als aktiv angesehen
 2. Konnte das Cursor-Objekt nicht erstellt werden, so wird der Fehler ausgegeben
2. Gib den Zustand der Verbindung aus

In [35]:
try:
    cur = con.cursor()
    connectionEstablished = True
except Exception as e:
    print(e)
print('Connection established: ' + str(connectionEstablished))

Connection established: True


---

## Erstellen eines Tables
1. Erstelle einen neuen Table
2. Frage alle Daten aus dem neuen Table ab (aktuell keine da leer)

In [36]:
# Constrain for "Id" has to be "INTEGER PRIMARY KEY"
# to result in auto-increment column
print(connectionEstablished)
if connectionEstablished:
    cur.execute("""CREATE TABLE IF NOT EXISTS person (
                   Id INTEGER PRIMARY KEY,
                   LastName TEXT NOT NULL,
                   FirstName TEXT,
                   DateOfBirth TEXT)""")
    con.commit()

    cur.execute("SELECT * FROM person")
    rows = cur.fetchall()
    if len(rows) > 0:
        for row in rows:
            print(row)
    else:
        print("No results")
    
else:
    print("Connection not established")


True
No results


---

\
Der Einfachheit halber wurden 2 Funktionen erstellt: `execute_and_fetch()` und `fetch()`\
Erstere führt eine übergebene Query aus, ruft die Daten ab und zeigt sie an.\
Zweitere ruft nur die Daten ab und zeigt sie an.

In [40]:
def execute_and_fetch(connobj, query, tablename):
    connectionEstablished = False
    try:
        cursor = connobj.cursor()
        connectionEstablished = True
    except Exception as e:
        print(e)
    
    if connectionEstablished:
        # Run query passed my sql argument against database
        try:
            cursor.execute(query)
            connobj.commit()
        except Exception as e:
            print(e)

        # Fetch current contents of table
        selquery = "SELECT * FROM " + str(tablename)
        try:
            cursor.execute(selquery)
        except Exception as e:
            print(e)
        
        rows  = cursor.fetchall()
        
        # Print results if there are any
        if len(rows) > 0:
            for row in rows:
                print(row)
        else:
            print("No results")
        
    else:
        print("Connection not established")
        
def fetch(connobj, tablename):
    connectionEstablished = False
    try:
        cursor = connobj.cursor()
        connectionEstablished = True
    except Exception as e:
        print(e)
    
    if connectionEstablished:
        # Fetch current contents of table
        selquery = "SELECT * FROM " + str(tablename)
        try:
            cursor.execute(selquery)
        except Exception as e:
            print(e)
        
        rows  = cursor.fetchall()
        
        # Print results if there are any
        if len(rows) > 0:
            for row in rows:
                print(row)
        else:
            print("No results")
        
    else:
        print("Connection not established")

\
Mittels der zuvor definierten Funktion `execute_and_fetch()` wird ein Eintrag mittels `insert` in den neuen Table `person` eingefügt und der Inhalt des Tables abgefragt und ausgegeben.
1. Definieren der Query als String und abspeichern in einer Variable (optional)
2. Aufrufen von `execute_and_fetch()` zur Ausführung der query und Abfrage der Daten

In [45]:
query = "INSERT INTO person VALUES (NULL, 'Baumann', 'Jan', '1900-01-01')"

execute_and_fetch(con, query, 'person')

(1, 'Baumann', 'Jan', '1900-01-01')
(2, 'Pavicic', 'Aleksandar', '1900-01-01')
(3, 'Pearson', 'Michael', '1900-01-01')
(4, 'Fahr', 'Alec', '1900-01-01')
(5, 'Baumann', 'Jan', '1900-01-01')


\
Um den Eintrag wieder zu löschen rufen wir die Funktion erneut auf, jedoch mit einer `delete` query.

In [41]:
query = "DELETE FROM person WHERE LastName = 'Baumann'"

execute_and_fetch(con, query, 'person')

No results


---

## Ausführen mehrere queries
Um mehrere Queries auszuführen können wir nicht den gleichen Befehl verwenden, da `execute()` nur eine einzelne Query erlaubt.

In [43]:
query1 = "INSERT INTO person VALUES (NULL, 'Baumann', 'Jan', '1900-01-01');"
query2 = "INSERT INTO person VALUES (NULL, 'Pavicic', 'Aleksandar', '1900-01-01');"
        
execute_and_fetch(con, query1 + query2, 'person')

You can only execute one statement at a time.
No results


\
Daher verwenden wir `executemany()` um mehrere queries nacheinander auszuführen.

In [44]:
query1 = "INSERT INTO person VALUES (NULL, 'Baumann', 'Jan', '1900-01-01');"
query2 = "INSERT INTO person VALUES (NULL,'Pavicic','Aleksandar','1900-01-01');"
query3 = "INSERT INTO person VALUES (NULL, 'Pearson', 'Michael', '1900-01-01');"
query4 = "INSERT INTO person VALUES (NULL, 'Fahr','Alec', '1900-01-01');"

cur.executescript(query1 + query2 + query3 + query4)
con.commit()

fetch(con, 'person')

(1, 'Baumann', 'Jan', '1900-01-01')
(2, 'Pavicic', 'Aleksandar', '1900-01-01')
(3, 'Pearson', 'Michael', '1900-01-01')
(4, 'Fahr', 'Alec', '1900-01-01')


---

Wir haben bisher nur `fetchall()` verwendet, um alle Resultate einer `SELECT` query aufzurufen.\
Wir erhalten dabei die Resultate immer als Liste. \
Form: `[<val 1>, <val 2>, ... <val n>]`\
\
Eine Liste ist eine der 4 Formen in Python, über welche mehrere Datensätze in einer einzelnen Variable abspeicherbar sind.
Die 4 Formen sind:
1. List (geordnet, veränderbar, indiziert, nicht eindeutig (erlaubt doppelte Werte))
2. Tuple (geordnet, unveränderbar, indiziert, nicht eundeutig)
3. Set (nicht geordnet, unveränderbar, nicht indiziert, eindeutig)
4. Dictionary (geordnet (ab Python 3.7), veränderbar, nicht numerisch indiziert (zugriff über Key-Value), eindeutig)

Nachfolgend werden mittels einer `SELECT` Abfrage zuerst alle Datensätze aus der Tabelle `person` ausgelesen.

In [49]:
cur.execute("SELECT * FROM person")

Die Resultate wurden nun vom Cursor gepuffert. Die Einträge in diesem Puffer können wir über `fetchall()` alle auf ein Mal abrufen und erhalten damit eine Liste mit mehreren Tuple.\
Jedes Tuple entspriche dabei einer Zeile, und enthält mehrere Werte.\
Das Tuple ist wie open erwähnt eine eine Auflistung mehrere Werte in Python, wobei sowohl die Reihenfolge als auch die einzelnen Datensätze unveränderbar sind.\
Form: `(<val 1>, <val 2>, ... <val n>)`
\
Mittels `fetchone()` können wir aber auch nur einzelne Tuple aus unserem Puffer auslesen. Für jeden Aufruf wird uns `fetchone()` dabei den nächsten Wert aus der Liste zurückgeben.\
`fetchone()` verwendet dabei bei jedem Aufruf den Generator der Liste, wodurch der jeweils nächste Wert aus der Liste zurückgegeben wird. Dazu später mehr.

In [54]:
print(cur.fetchone())

None


\
Mittels dem SQL Befehl `COUNT()` und `fetchone()` können wir nun die Anzahl der Einträge in unserem Table auslesen.
1. Ausführen der Query gegen die Datenbank
2. Abfragen des Resultates (COUNT() liefert in diesem Fall nur 1 Resultat)
3. Abspeichern des Resultats in einer Variable, Zugriff auf das Tuple wie auf ein Array
4. Ausgabe der Anzahl

In [46]:
cur.execute("SELECT COUNT() FROM person")
data = cur.fetchone()
numrows = data[0]
print("Number of rows: " + str(numrows))

Number of rows: 5


\
Wir können dieses Prinzip verwenden, um eine gewisse Anzahl von Zeilen zu löschen. Hierfür können wir `executemany()` verwenden.\
`executemany()` erwartet als zweiten Parameter einen Generator. Dies ist eine Spezialform einer Funktion, welche einen Iterator implementiert.\
Ein Iterator ist eine spezielles Objekt, welches für jeden Aufruf einen Wert aus einer Auflistung zurückgibt.\
\
Im Fall hier gibt der Generator `idGen` für jeden Aufruf einen Wert im Bereich zwischen den Parametern `min` und `max` (beide inklusive) aus.\
`executemany()` führt also die übergebene Query mit einem Platzhalter für den Iterable so lange aus, wie der Generator einen Wert zurückliefert.\
In der Implementation unten also für alle IDs zwischen 2 und der Anzahl der Zeilen in der Tabelle.

1. Definition des Generators `idGen`
2. Ausführen der gleichen Query wie oben zum Erhalt der Anzahl Zeilen in der Tabelle und abspeichern dieser in einer Variable
3. Ausführen der Query in `executemany()` unter Zuhilfename des Generators für alle IDs
4. Abfragen der noch vorhandenen Datensätze über `fetch()`

*Anmerkung: Es wäre auch möglich gewesen, mittels `SELECT` alle in der Tabelle vorhandenen IDs abzufragen und mittels einem Generator die IDs aus dem Tuple ab der zweiten ID zurückzugeben, aber das hätte die Sache weiter verkompliziert und der Demonstration keinen Mehrwert gebracht.*

In [56]:
def idGen(min, max):
    for i in range(min, max+1):
        yield (i,)

cur.execute("SELECT COUNT(*) FROM person")
numrows = cur.fetchone()[0]
print("Number of rows: " + str(numrows))

cur.executemany("DELETE FROM person WHERE Id == (?)", idGen(2, numrows))
con.commit()

fetch(con, 'person')

Number of rows: 7
(1, 'Baumann', 'Jan', '1900-01-01')


\
Eine Andere Anwendung wäre, dass wir direkt ein Tuple Ojekt angeben, da Tuple das Interface `Iterable` implementiert und damit einen Generator bereitstellt, welchen `executemany()` ja als Argument erwartet.\
Ein Interface definiert die von einem Objekt zu implementierenden Methoden und erlaubt daher die Verwendung die Verwendung von definierten Methoden, mehr dazu in OOP2 im Kapitel "Interfaces"
1. "Abfüllen" des Tuple mit den entsprechenden Werten
2. Ausführen der Query `executemany()` mit den Platzhaltern für die Werte, sowie dem Tuple als Argument
3. Abfragen der Datensätze

In [57]:
values = [(None, "Pavicic""", "Aleksandar""", "1900-01-01"),
          (None, "Pearson", "Michael""", "1900-01-01"),
          (None, "Fahr", "Alec", "1900-01-01")]
cur.executemany("INSERT INTO person VALUES (?, ?, ?, ?)", values)
con.commit()

fetch(con, 'person')

(1, 'Baumann', 'Jan', '1900-01-01')
(2, 'Pavicic', 'Aleksandar', '1900-01-01')
(3, 'Pearson', 'Michael', '1900-01-01')
(4, 'Fahr', 'Alec', '1900-01-01')


---

Mit Hilfe von `backup()` können wir ein Backup der bestehenden Datenbank erstellen.\
Dabei muss als erstes Argument ein `Connection` Objekt zu der Backup-Datenbank übergeben werden.\
Der zweite Parameter definiert die Anzahl Seiten, jede Seite umfasst standardmässig 2^10, also 1024 Bytes.\
Über den dritten Parameter kann eine Funktion angegeben werden, welche als Argumente 3 Werte erhält, welche Auskunft über den Fortschritt des Backup-Prozesses ausgeben.

In [64]:
conbackup = sqlite3.connect('data/backup.db')

print("Person table current state: ")
fetch(con, 'person')

con.backup(conbackup, pages = 1, progress = (lambda status, remaining, total : print(f"Copied {total - remaining} of {total} pages...")))

print("\nPerson table from backup: ")
fetch(conbackup, 'person')

Person table current state: 
(1, 'Baumann', 'Jan', '1900-01-01')
(2, 'Pavicic', 'Aleksandar', '1900-01-01')
(3, 'Pearson', 'Michael', '1900-01-01')
(4, 'Fahr', 'Alec', '1900-01-01')
Copied 1 of 2 pages...
Copied 2 of 2 pages...

Person table from backup: 
(1, 'Baumann', 'Jan', '1900-01-01')
(2, 'Pavicic', 'Aleksandar', '1900-01-01')
(3, 'Pearson', 'Michael', '1900-01-01')
(4, 'Fahr', 'Alec', '1900-01-01')


\
Nachdem wir nun ein Backup der Datenbank erstellt haben, können wir den Table löschen.\
Wenn wir nun `fetch()` aufrufen erhalten wir eine Fehlermeldung.

In [24]:
cur.execute("DROP TABLE IF EXISTS person")
con.commit()

fetch(con, 'person')

no such table: person


\
Wenn alles gut gelaufen ist, so ist der Table aber immer noch in unsere Backup-Datenbank vorhanden.

In [65]:
fetch(conbackup, 'person')

(1, 'Baumann', 'Jan', '1900-01-01')
(2, 'Pavicic', 'Aleksandar', '1900-01-01')
(3, 'Pearson', 'Michael', '1900-01-01')
(4, 'Fahr', 'Alec', '1900-01-01')


\
Wir schliessen die Verbindung zur Backup-Datenbank, da sie nicht mehr benötigt werd. Dazu später mehr.

In [67]:
conbackup.close()

---

Wenn wir alle Queries gegen die Datenbank durchgeführt haben, so können wir das `Cursor` Objekt schliessen und damit alle Operationen an der Datenbank abschliessen.\
Dies wird mittels `close()` durchgeführt. Dabei gibt es `close` sowohl für das `Cursor`, als auch das `Connection` Objekt.\
Nach dem Aufruf von `close()` auf dem `Cursor` Objekt ist dieses nicht mehr für weitere Queries verfügbar. Wird trotzdem versucht, eine Query auszuführen, so tritt ein Fehler auf.\
__Alle noch offenen Änderungen, welche nicht mit `commit()` bestätigt wurden, werden mit dem schliessen des `Cursor` Objektes verworfen.__

In [68]:
cur.close()

try:
    cur.execute("SELECT * FROM person")
except Exception as e:
    print(e)

Cannot operate on a closed cursor.


\
Wie erwähnt existiert die `close()` Methode nicht nur für das `Cursor` Objekt, sondern auch für das `Connection` Objekt.\
Mit dem schliessen der Verbindung zur Datenbank werden alle `Cursor` Objekte gelöscht und die Verbindung zur Datenbank getrennt.\
Die Datenbank ist somit für andere Anwendungen freigegeben.

In [69]:
con.close()

try:
    con.cursor()
except Exception as e:
    print(e)

Cannot operate on a closed database.


Grundsätzlich sollte nach allen __Änderungen__ die `commit()` Methode auf das `Cursor` Objekt, und nach der Beendigung aller benötigten Queries die `close()` auf das `Cursor` und das `Connection` Objekt angewendet werden, um Konflikte zu vermeiden